In [1]:
import gzip
import opener
import pandas as pd
import io
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from collections import Counter

In [2]:
url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz"
req = requests.get(url)

In [3]:
t = req.content

In [4]:
s = gzip.decompress(t)

In [5]:
df = pd.read_csv(io.BytesIO(s))

C:\Users\dargo\AppData\Local\Temp\ipykernel_12224\798997524.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.BytesIO(s))


In [6]:
df

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Appartement,73.0,4.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
1,2018-1,2018-01-03,1,Vente,109000.0,13.0,NaN,RUE GEN LOGEROT,1660,1000.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.220440,46.200062
2,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,S,sols,NaN,NaN,949.0,5.408041,46.255562
3,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,163.0,4.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
4,2018-2,2018-01-04,1,Vente,239300.0,4.0,NaN,RUE DE LA BARMETTE,0025,1250.0,...,Maison,51.0,2.0,AG,terrains d'agrément,JARD,Jardin d'agrément,420.0,5.408041,46.255562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335287,2018-1434017,2018-12-28,1,Vente,1192307.0,1.0,NaN,PAS CHOISEUL,2012,75002.0,...,Appartement,150.0,4.0,NaN,NaN,NaN,NaN,NaN,2.335139,48.867253
3335288,2018-1434018,2018-12-03,1,Vente,383000.0,12.0,NaN,RUE POISSONNIERE,7561,75002.0,...,Appartement,34.0,1.0,NaN,NaN,NaN,NaN,NaN,2.347989,48.869018
3335289,2018-1434019,2018-12-28,1,Vente,746000.0,18.0,NaN,RUE STE CROIX BRETONNERIE,8548,75004.0,...,Local industriel. commercial ou assimilé,60.0,0.0,NaN,NaN,NaN,NaN,NaN,2.356566,48.858267
3335290,2018-1434020,2018-12-13,1,Adjudication,645000.0,9.0,NaN,BD MORLAND,6559,75004.0,...,Appartement,54.0,2.0,NaN,NaN,NaN,NaN,NaN,2.364563,48.848333


In [129]:
# suppression valeur fonciere nulle

df.dropna(subset=['valeur_fonciere'], inplace=True)
df.dropna(subset=['longitude', 'latitude'], inplace=True)

In [130]:
# Selection des colonnes utiles

dataset = df[['id_mutation',
              'date_mutation',
              'nature_mutation',
              'adresse_nom_voie',
              'code_postal',
              'code_commune',
              'code_departement',
              'nombre_lots',
              'type_local',
              'surface_reelle_bati',
              'nombre_pieces_principales',
              'code_nature_culture',
              'code_nature_culture_speciale',
              'surface_terrain',
              'valeur_fonciere',
              'longitude',
              'latitude'
              ]]

In [131]:
# del df

# Cleaning

In [132]:
dataset.shape

(3201638, 17)

### Date mutation cleaning

In [133]:
# Passage de la colonnes date mutation en datetime puis en seconde pour avoir des int

dataset.loc[:, 'date_mutation'] = pd.to_datetime(dataset['date_mutation'], format='mixed')
dataset.loc[:, 'date_mutation'] = dataset['date_mutation'].apply(lambda x: x.timestamp())

In [134]:
# Remplissage des valeurs NaN nature_mutation et labelisation

dataset.loc[:, 'nature_mutation'] = dataset.nature_mutation.fillna("")

NM = preprocessing.LabelEncoder()
NM.fit(dataset['nature_mutation'])
dataset.loc[:, 'nature_mutation'] = NM.transform(dataset['nature_mutation'])

### Adresse nom voie cleaning

In [135]:
abrv_voie = pd.read_csv(filepath_or_buffer='Data_Files/ABREVIATION_VOIE.csv', sep=',').values

In [136]:
dataset.loc[:, 'adresse_nom_voie'] = dataset.adresse_nom_voie.fillna("")

codex_voie = list()
for i in dataset.adresse_nom_voie.values:

    list_nom_rue = i.split(' ')
    premier_valeur_liste = list_nom_rue[0]

    if premier_valeur_liste in abrv_voie:
        codex_voie.append(premier_valeur_liste)
    else:
        codex_voie.append('AUTRE')

dataset.loc[:, 'prefixe_voie'] = codex_voie

C:\Users\dargo\AppData\Local\Temp\ipykernel_12224\2922452403.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'prefixe_voie'] = codex_voie


CV = preprocessing.LabelEncoder()
CV.fit(dataset.prefixe_voie)
dataset.loc[:,'prefixe_voie'] = CV.transform(dataset.prefixe_voie)

### Multiple easy Cleaning

In [137]:
dataset.loc[:, 'code_postal'] = dataset.code_postal.fillna(0)

In [139]:
dataset.loc[:, 'surface_reelle_bati'] = dataset.surface_reelle_bati.fillna(0)
dataset.loc[:, 'surface_terrain'] = dataset.surface_terrain.fillna(0)
dataset.loc[:, 'type_local'] = dataset.type_local.fillna('Autre')
dataset.loc[:, 'nombre_pieces_principales'] = dataset.nombre_pieces_principales.fillna(0)

In [140]:
dataset.loc[:, 'code_nature_culture'] = dataset.code_nature_culture.fillna("")
dataset.loc[:, 'code_nature_culture_speciale'] = dataset.code_nature_culture_speciale.fillna("")

### Latitude & longitude

In [141]:
dataset.longitude.isna().sum()

0

In [142]:
dataset.latitude.isna().sum()

0

In [143]:
dataset.dtypes

id_mutation                      object
date_mutation                    object
nature_mutation                  object
adresse_nom_voie                 object
code_postal                     float64
code_commune                     object
code_departement                 object
nombre_lots                       int64
type_local                       object
surface_reelle_bati             float64
nombre_pieces_principales       float64
code_nature_culture              object
code_nature_culture_speciale     object
surface_terrain                 float64
valeur_fonciere                 float64
longitude                       float64
latitude                        float64
prefixe_voie                     object
dtype: object

# Construction data set modele

In [144]:
classe_liste_nature_mutation = list(dict.fromkeys(list(dataset.nature_mutation.values)))

In [145]:
classe_liste_code_type_local = list(dict.fromkeys(list(dataset.type_local.values)))

In [146]:
classe_liste_prefixe_voie = list(dict.fromkeys(codex_voie))

In [147]:
classe_liste_code_culture = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE.csv', sep=',')
classe_liste_code_culture = list(classe_liste_code_culture['Code_nature_culture'].to_dict().values())

In [148]:
classe_liste_code_culture_spe = pd.read_csv(filepath_or_buffer='Data_Files/CODE_CULTURE_SPECIALE.csv', sep=',')
classe_liste_code_culture_spe = list(classe_liste_code_culture_spe['CODE_CULTURE_SPECIALE'].to_dict().values())

In [149]:
Nom_colonnes_from_dataset = [
    'date_mutation',
    'code_postal',
    'code_commune',
    'code_departement',
    'nombre_lots',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'surface_terrain',
    'valeur_fonciere',
    'longitude',
    'latitude'
]

In [150]:
colonnes = Nom_colonnes_from_dataset + classe_liste_code_type_local + classe_liste_prefixe_voie + classe_liste_nature_mutation +  classe_liste_code_culture + classe_liste_code_culture_spe

In [151]:
dataset.shape

(3201638, 18)

In [152]:
index =123456

In [153]:
test_id = dataset.id_mutation[index]

In [154]:
test_id

'2018-51207'

In [155]:
TEST = dataset.loc[dataset['id_mutation'] == test_id]

In [156]:
assert 1 == len(pd.unique(TEST.date_mutation))
assert 1 == len(pd.unique(TEST.valeur_fonciere))
assert 1 == len(pd.unique(TEST.code_postal))
assert 1 == len(pd.unique(TEST.code_commune))
assert 1 == len(pd.unique(TEST.code_departement))

ajout_colonnes_dataset = [
    TEST.loc[index, 'date_mutation'],
    TEST.loc[index, 'code_postal'],
    TEST.loc[index, 'code_commune'],
    TEST.loc[index, 'code_departement'],
    TEST.loc[:, 'nombre_lots'].sum(),
    TEST.loc[:,'surface_reelle_bati'].sum(),
    TEST.loc[:,'nombre_pieces_principales'].sum(),
    TEST.loc[:,'surface_terrain'].sum(),
    TEST.loc[index, 'valeur_fonciere'],
    TEST.loc[index, 'longitude'],
    TEST.loc[index, 'latitude']
]

In [157]:
# valeur classe code type local

tmp_code_type_local = TEST.loc[:,'type_local']
ajout_code_type = []

for code_type in classe_liste_code_type_local :
    ajout_code_type.append(tmp_code_type_local.loc[tmp_code_type_local == code_type].size)


In [171]:
classe_liste_code_type_local

In [159]:
assert len(ajout_code_type) == len(classe_liste_code_type_local)

In [160]:
# valeur prefixe voie

tmp_abbrev_voie = TEST.loc[:, 'prefixe_voie']
ajout_abbrev_voie = []

for abbrev_voie in classe_liste_prefixe_voie:
    ajout_abbrev_voie.append(tmp_abbrev_voie.loc[tmp_abbrev_voie== abbrev_voie].size)

In [161]:
assert len(ajout_abbrev_voie) == len(classe_liste_prefixe_voie)

In [162]:
# valeur classe nature mutation

tmp_nature_mutation = TEST.loc[:,'nature_mutation']
ajout_nature_mutation = []

for nat_mutation in classe_liste_nature_mutation:
    ajout_nature_mutation.append(tmp_nature_mutation.loc[tmp_nature_mutation == nat_mutation].size)

In [163]:
assert len(ajout_nature_mutation) == len(classe_liste_nature_mutation)

In [164]:
# valeur classe culture

tmp_culture = TEST.loc[:,'code_nature_culture']
ajout_culture = []

for culture in classe_liste_code_culture :
    ajout_culture.append(tmp_culture.loc[tmp_culture == culture].size)


In [165]:
assert len(ajout_culture) == len(classe_liste_code_culture)

In [166]:
# valeur classe culture spe

tmp_culture_spe = TEST.loc[:,'code_nature_culture_speciale']
ajout_culture_spe = []

for culture_spe in classe_liste_code_culture_spe :
    ajout_culture_spe.append(tmp_culture_spe.loc[tmp_culture_spe == culture_spe].size)

In [167]:
assert len(ajout_culture_spe) == len(classe_liste_code_culture_spe)

In [168]:
valeurs = ajout_colonnes_dataset + ajout_code_type + ajout_abbrev_voie + ajout_nature_mutation + ajout_culture + ajout_culture_spe

colonnes = Nom_colonnes_from_dataset + classe_liste_code_type_local + classe_liste_prefixe_voie + classe_liste_nature_mutation +  classe_liste_code_culture + classe_liste_code_culture_spe

In [169]:
tmp_data = { colonnes[i]:valeurs[i] for i in range(len(colonnes)) }
tmp_data

{'date_mutation': 1539907200.0,
 'code_postal': 6200.0,
 'code_commune': 6088,
 'code_departement': 6,
 'nombre_lots': 0,
 'surface_reelle_bati': 80.0,
 'nombre_pieces_principales': 6.0,
 'surface_terrain': 6437.0,
 'valeur_fonciere': 300000.0,
 'longitude': 7.214999,
 'latitude': 43.700335,
 'Appartement': 0,
 'Dépendance': 0,
 'Maison': 2,
 '': 1,
 'Local industriel. commercial ou assimilé': 0,
 'RUE': 0,
 'AUTRE': 3,
 'PONT': 0,
 'ALL': 0,
 'AV': 0,
 'RTE': 0,
 'PRE': 0,
 'PL': 0,
 'QUAI': 0,
 'BD': 0,
 'IMP': 0,
 'CHEM': 0,
 'LOT': 0,
 'A': 0,
 'BOIS': 0,
 'CLOS': 0,
 'PROM': 0,
 'COTE': 0,
 'VC': 0,
 'RLE': 0,
 'CRS': 0,
 'MONT': 0,
 'MARE': 0,
 'HAM': 0,
 'ILE': 0,
 'MAIL': 0,
 'FOND': 0,
 'RES': 0,
 'VOIE': 0,
 'CITE': 0,
 'ST': 0,
 'ZA': 0,
 'PTR': 0,
 'ZI': 0,
 'CHAT': 0,
 'PORT': 0,
 'TOUR': 0,
 'PARC': 0,
 'SQ': 0,
 'RPE': 0,
 'CR': 0,
 'ESC': 0,
 'CD': 0,
 'DOM': 0,
 'GARE': 0,
 'VAL': 0,
 'FONT': 0,
 'FBG': 0,
 'CHP': 0,
 'PASS': 0,
 'ESP': 0,
 'FORT': 0,
 'VALL': 0,
 'VEN

In [175]:
tmp_Data_Frame = pd.DataFrame(tmp_data, index=[0])
tmp_Data_Frame

,date_mutation,code_postal,code_commune,code_departement,nombre_lots,surface_reelle_bati,nombre_pieces_principales,surface_terrain,valeur_fonciere,longitude,...,TOURB,TOUYA,VADC,VAGUE,VANIL,VAOC,VCHAS,VDQS,VIGNE,VIVIE
0,1.539907e+09,6200.0,6088,6,0,80.0,6.0,6437.0,300000.0,7.214999,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Sortie de la colonne resultat

## Y = df.valeur_fonciere